# **Lab2: RS by model-based**


## **Matrix Factorization by using SVD**

Last week we directly filled the missing rating data with zero to induce the  sparse matrix (i.e. Tf-Idf, userId-movieId). 

However, we could also compute an estimated userId-movieId matrix with SVD in an iterative learning process. Here, we use the known ratings and try to minimize the error of computing the known rates via gradient descent.

### **Mathematical Description**

SVD factorizes our feature matrix $M_{m\times n}$ with a rank of $k$,according to equation **(1a)** with 3 matrices of $U_{m\times k}$ ,$\sum_{k\times k}$ and $𝐼^𝑇_{n \times 𝑘}$:

$$M=U\sum_{k} I^T \tag{1a}$$

$$M \approx U\sum_{k^{'}} I^T \tag{1b}$$

where $U$ is the matrix of user preferences, $I$ is the item preferences and $\sum$ is the matrix of singular values.

The beauty of SVD in this simple notion that instead of full $k$ vector space, we can approximate our feature matrix $M$ on a much smaller $k^{'}$ latent space as in equation **(1b)**.

This approximation will reduce both the dimensions of the feature matrix, but it also takes into account only the most important singular values and leaves behind the smaller singular values which could otherwise result in noise.

To approximate $M$, we would like to find $U$ and $I$ matrices in $k^{'}$ space, which means to find the optimized $k^{'}$. According to equation **(2)**, every rate entry in $M$, $r_{ui}$ can be written as a dot product of $p_u$ and $q_i$:

$$r_{ui}=p_u \cdot q_i \tag{2}$$

where $p_u$ makes up the rows of $U$ considered as **user factors** and $q_i$ the columns of $I^T$ indicated as **item factors**. Here we disregard the diagonal matrix $\sum$ for simplicity (as it provides only a scaling factor). Graphically it would look something like this:

<p align="center">
  <img src="https://blog.codecentric.de/files/2019/06/Screenshot-2019-05-28-at-15.22.31.png"  width="400">
</p>

Finding all $p_u$ and $q_i$ for all users and items will be possible via the following optimization problem:

$$\min_{\{p_u, q_i\}}=\sum_{r_{ui} \in M}(r_{ui}-p_u \cdot q_i)^2 \tag{3}$$

Here, the optimization problem can be solved by a **gradient descent (GD) algorithm** (or a variant of it such as **stochastic gradient descent (SGD)**) can be used and to compute all $p_u$ and $q_i$. For more details you can read more about it [Here](http://nicolas-hug.com/blog/matrix_facto_1). After we have all the entries of $U$ and $I$, the predicted rating $\hat{r}_{ui}$ can be computed according to equation **(2)**:

$$\hat{r}_{ui} = q_i^Tp_u \tag{4}$$

The minimization process in equation **(3)** can also be regularized and fine-tuned with biases. [Read more](https://surprise.readthedocs.io/en/stable/matrix_factorization.html)

## **Movielens-100K Dataset Information**
*   **movies.csv**
    * movieId, title, genres
    * 9742 movies 

*   **ratings.csv**
    * userId, movieId, rating, timestamp
    * 100836 rows, 9724 unique movies, 610 unique users
*   **tags.csv**
    * userId, movieId, tag, timestamp
    * 3683 rows
*   **links.csv**
    * movieId, imdbId, tmdbId
    * 9742 rows


In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [0]:
# Upload your file to your Google Drive folder and load it 
# (It is required to change direction to the repository)
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/Teaching/Web Mining/2020/tutorials/RS"
%cd /content/drive/My\ Drive/Teaching/Web\ Mining/2020/tutorials/RS/ml-latest-small/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'Lab1_ RS for movies.ipynb'  'Lab2_ RS by using MF.ipynb'   ml-latest-small
/content/drive/My Drive/Teaching/Web Mining/2020/tutorials/RS/ml-latest-small


In [0]:
## Load movies data
movies = pd.read_csv('movies.csv')
tags = pd.read_csv('tags.csv') 
ratings = pd.read_csv('ratings.csv')
links = pd.read_csv('links.csv')

In [0]:
# Since currently we're using a rather small dataset, this won't be a case, so we can set the user ratings to 0
ratings_f = ratings.groupby('userId').filter(lambda x: len(x) >= 0)
print(len(ratings_f.groupby(['userId']).size()))
movie_list_rating = ratings_f.movieId.unique().tolist()
movies_filter = movies[movies.movieId.isin(movie_list_rating)]
print(len(movies_filter))
print('Preserved rate of the movies : {0:.2f} %'.format(len(ratings_f.movieId.unique())/len(movies_filter.movieId.unique()) * 100))
print('Preserved rate of the users : {0:.2f} %'.format(len(ratings_f.userId.unique())/len(ratings.userId.unique()) * 100))

610
9724
Preserved rate of the movies : 100.00 %
Preserved rate of the users : 100.00 %


In [0]:
# Map movie to id:
Mapping_file = dict(zip(movies_filter.title.tolist(), movies_filter.movieId.tolist()))
# Drop timestamp since we don't need it
tags.drop(['timestamp'],1, inplace=True)
ratings_f.drop(['timestamp'],1, inplace=True)

In [0]:
print('Total rows: ', len(ratings_f))
ratings_f.head(3)

Total rows:  100836


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0


## **Implementation via Surprise**

Here we use the [**Surprise**](http://surpriselib.com/) library. 

<p align="left">
  <img src="https://github.com/NicolasHug/Surprise/raw/master/logo_black.svg?sanitize=true"  width="400">
</p>

**Surprise** is a Python scikit building and analyzing recommender systems that deal with explicit rating data. Provide various ready-to-use prediction algorithms such as baseline algorithms, neighborhood methods, matrix factorization-based (SVD, NMF), and many others. Also, various similarity measures (cosine, MSD, pearson) are built-in.

In [0]:
# Install the surprise package
!pip install scikit-surprise

In [0]:
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split
# Instantiate a reader with the specified rating scale
# and read in the rating data
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(ratings_f[['userId','movieId','rating']], reader)
 
# Train SVD on 75% of known rates
trainset, testset = train_test_split(data, test_size=0.25, random_state = 0)

In [0]:
# Check the testset
df_test = pd.DataFrame(testset, columns=['userId', 'movieId', 'rating'])
df_test.head(3)

,userId,movieId,rating
0,63,2000,3.0
1,31,788,2.0
2,159,6373,4.0


In [0]:
# Train the model on trainset and evaluate on testset
# About the implemented SVD: https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD
algorithm = SVD()
model_svd = algorithm.fit(trainset)
predictions = model_svd.test(testset)

In [0]:
df_pred = pd.DataFrame(predictions, columns=['userId', 'movieId', 'rating', 'pred_rating', 'details'])
df_pred.head(5)

,userId,movieId,rating,pred_rating,details
0,63,2000,3.0,3.558195,{'was_impossible': False}
1,31,788,2.0,3.090127,{'was_impossible': False}
2,159,6373,4.0,2.992613,{'was_impossible': False}
3,105,81564,3.0,4.142113,{'was_impossible': False}
4,394,480,3.0,3.288856,{'was_impossible': False}


In [0]:
# Check the accuracy using Root Mean Square Error
print(accuracy.rmse(predictions))

RMSE: 0.8699
0.8699030611309359


In [0]:
# Check the accuracy using Mean Absolute Error
print(accuracy.mae(predictions))

MAE:  0.6674
0.6673569638330299


### **Discussion**
* What's the potential problem here?

### **Cross-validation**

In [0]:
from sklearn.model_selection import train_test_split
from surprise.model_selection import cross_validate
reader = Reader(rating_scale=(0, 5))
trainset, testset = train_test_split(ratings_f[['userId','movieId','rating']], test_size=0.25, random_state=0)
trainset.index = range(len(trainset)) # 75627
testset.index = range(len(testset)) # 25209
trainset = Dataset.load_from_df(trainset, reader)
#data = Dataset.load_from_df(ratings_f[['userId','movieId','rating']], reader)
#trainset = Dataset.load_from_df(trainset, reader).build_full_trainset()
#testset = Dataset.load_from_df(testset, reader)
testset = testset.values.tolist()
benchmark = []
i = 0
alg_list = ['SVD(n_factors=50)', 'SVD(n_factors=100)', 
            'SVD(n_factors=150)']
for algorithm in [SVD(n_factors=50), SVD(n_factors=100), SVD(n_factors=150)]:
    # Perform cross validation
    results = cross_validate(algorithm, trainset, measures=['RMSE','MAE'], cv=5, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([alg_list[i]], index=['Algorithm']))
    benchmark.append(tmp)
    i+=1
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

,test_rmse,test_mae,fit_time,test_time
Algorithm,,,,
SVD(n_factors=50),0.879632,0.677314,2.567455,0.135399
SVD(n_factors=100),0.883435,0.679918,4.066330,0.161738
SVD(n_factors=150),0.886776,0.682871,5.416952,0.144465


In [0]:
# Train on full training set and evaluate on test set
trainset = trainset.build_full_trainset()
algorithm = SVD(n_factors=50)
model_svd = algorithm.fit(trainset)
predictions = model_svd.test(testset)

In [0]:
print(accuracy.rmse(predictions))
print(accuracy.mae(predictions))

RMSE: 0.8783
0.8782847611881339
MAE:  0.6741
0.6741170733692327


## **How about other algorithms?**



* **Non-negative Matrix Factorization (NMF)**
  * An algorithm similar to SVD, while forcing the decomposed matrix $U$ and $I$ having non-negative values. This means that the user factor $p_u$ and the item factor $q_i$ are kept positive. (i.e. the predicted rating should be positive since the rating scale is between 0 to 5)
  * Select the optimized factor (i.e. $k^{'}$), the default value in Surprise library is 15
  $$\hat{r}_{ui} = q_i^Tp_u$$

* **KNN**
  * A baseline approach to  measure the  similarities  between the rating patterns of items. KNN does not make any assumptions on the underlying data distribution but it relies on item feature similarity. When KNN makes inference about a movie, KNN will calculate the **distance** between the target movie and every other movie in its database, then it ranks its distances and returns the top K nearest neighbor movies as the most similar movie recommendations.
  * **KNNBasic**
    * User-based
  $$\hat{r}_{ui} = \frac{
\sum\limits_{v \in N^k_i(u)} \text{sim}(u, v) \cdot r_{vi}}
{\sum\limits_{v \in N^k_i(u)} \text{sim}(u, v)}$$
    * Item-based
  $$\hat{r}_{ui} = \frac{
\sum\limits_{j \in N^k_u(i)} \text{sim}(i, j) \cdot r_{uj}}
{\sum\limits_{j \in N^k_u(i)} \text{sim}(i, j)}$$

  * **KNNWithMeans**
    * User-based
  $$\hat{r}_{ui} = \mu_u + \frac{ \sum\limits_{v \in N^k_i(u)}
\text{sim}(u, v) \cdot (r_{vi} - \mu_v)} {\sum\limits_{v \in
N^k_i(u)} \text{sim}(u, v)}$$
    * Item-based
  $$\hat{r}_{ui} = \mu_i + \frac{ \sum\limits_{j \in N^k_u(i)}
\text{sim}(i, j) \cdot (r_{uj} - \mu_j)} {\sum\limits_{j \in
N^k_u(i)} \text{sim}(i, j)}$$

[More to discover](https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html)


In [0]:
from surprise import SVD, NMF, KNNBasic, KNNWithMeans
from sklearn.model_selection import train_test_split

## Load the data into surprise input data format, split into training and testing
reader = Reader(rating_scale=(0, 5))
trainset, testset = train_test_split(ratings_f[['userId','movieId','rating']], test_size=0.25, random_state=0)
trainset.index = range(len(trainset)) # 75627
testset.index = range(len(testset)) # 25209
trainset = Dataset.load_from_df(trainset, reader)
testset = testset.values.tolist()

# Define the models and their hyperparameters 
sim_option_item = {'name': 'cosine',
              'user_based': False  # compute similarities between items
               }
sim_option_user = {'name': 'pearson',
              'user_based': True  # compute similarities between users
               }
benchmark = []
i = 0
alg_list = ['SVD(n_factors=50)', 'NMF(n_factors=15)', 
            'KNNBasic(k=20, sim=cosine, item-based)', 
            'KNNBasic(k=20, sim=pearson, user-based)']

# Iterate over selected algorithms based on 5-fold CV on training set
for algorithm in [SVD(n_factors=50), NMF(n_factors=15), KNNBasic(k=20, sim_options=sim_option_item), KNNBasic(k=20, sim_options=sim_option_user)]:
    # Perform cross validation
    results = cross_validate(algorithm, trainset, measures=['RMSE','MAE'], cv=5, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([alg_list[i]], index=['Algorithm']))
    benchmark.append(tmp)
    i+=1
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')    

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


,test_rmse,test_mae,fit_time,test_time
Algorithm,,,,
SVD(n_factors=50),0.879445,0.676697,2.585811,0.129712
NMF(n_factors=15),0.941277,0.722808,4.866181,0.116344
"KNNBasic(k=20, sim=cosine, item-based)",0.988901,0.770856,13.337167,4.569720
"KNNBasic(k=20, sim=pearson, user-based)",0.991845,0.765765,0.374504,0.891866


In [0]:
# Since SVD(n_factors=50) reach the lowest RMSE and MAE
# We use this model to train on whole training set and evaluate the final result on testset
trainset = trainset.build_full_trainset()
algorithm = SVD(n_factors=50)
model_svd = algorithm.fit(trainset)
predictions = model_svd.test(testset)
print(accuracy.rmse(predictions))
print(accuracy.mae(predictions))

RMSE: 0.8786
0.8786352440622177
MAE:  0.6745
0.6745174695626888


## **Final RS based on the selected algorithm**

In [0]:
def pred_user_rating(userid, num_recommend):
    if userid in ratings_f.userId.unique():
        ui_list = ratings_f[ratings_f.userId == userid].movieId.tolist()
        d = {k: v for k,v in Mapping_file.items() if not v in ui_list}        
        predictedL = []
        for i, j in d.items():     
            predicted = algorithm.predict(userid, j)
            predictedL.append((i, predicted[3])) 
        pdf = pd.DataFrame(predictedL, columns = ['movies', 'ratings'])
        pdf.sort_values('ratings', ascending=False, inplace=True)  
        pdf.set_index('movies', inplace=True)    
        return pdf.head(num_recommend)        
    else:
        print("User Id does not exist in the list!")
        return None

In [0]:
user_id = int(input("Enter the user id to whom you want to recommend : "))
num_recommend = int(input("Enter the number of movies you want to recommend : "))
pred_user_rating(user_id, num_recommend)

Enter the user id to whom you want to recommend : 2
Enter the number of movies you want to recommend : 10


,ratings
movies,
Casablanca (1942),4.478652
Star Wars: Episode V - The Empire Strikes Back (1980),4.461504
"Grand Day Out with Wallace and Gromit, A (1989)",4.458610
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.456449
Ghost in the Shell (Kôkaku kidôtai) (1995),4.429836
Rear Window (1954),4.421447
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),4.420310
Cool Hand Luke (1967),4.394462
In Bruges (2008),4.380208


In [0]:
print(len(ratings_f[ratings_f.userId==2]))
ratings_f[ratings_f.userId==2].merge(movies).sort_values('rating', ascending=False)

29


,userId,movieId,rating,title,genres
28,2,131724,5.0,The Jinx: The Life and Deaths of Robert Durst ...,Documentary
27,2,122882,5.0,Mad Max: Fury Road (2015),Action|Adventure|Sci-Fi|Thriller
22,2,106782,5.0,"Wolf of Wall Street, The (2013)",Comedy|Crime|Drama
18,2,89774,5.0,Warrior (2011),Drama
9,2,60756,5.0,Step Brothers (2008),Comedy
16,2,80906,5.0,Inside Job (2010),Documentary
2,2,1704,4.5,Good Will Hunting (1997),Drama|Romance
8,2,58559,4.5,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX
10,2,68157,4.5,Inglourious Basterds (2009),Action|Drama|War
15,2,80489,4.5,"Town, The (2010)",Crime|Drama|Thriller


In [0]:
trainset, testset = train_test_split(ratings_f[['userId','movieId','rating']], test_size=0.25, random_state=0)
trainset.index = range(len(trainset)) # 75627
testset.index = range(len(testset))
print(len(testset[testset.userId==2]))
check = testset[testset.userId==2]
check.index = range(len(check))
check.merge(movies)

3


,userId,movieId,rating,title,genres
0,2,58559,4.5,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX
1,2,80906,5.0,Inside Job (2010),Documentary
2,2,115713,3.5,Ex Machina (2015),Drama|Sci-Fi|Thriller


## **Other useful libraries**

**1. FastAI**
  
  
  Similar with scikit-learn, [FastAI](https://www.fast.ai/) has the mission of making deep learning easier to use and getting more people from all backgrounds involved. If you're interested in trying other libraries, feel free to check the following sources and get more insights into it. It might be helpful to your project or future work!

  * [Deep Learning for Collaborative Filtering (using FastAI)](https://medium.com/quantyca/deep-learning-for-collaborative-filtering-using-fastai-b28e197ccd59)
  * [Neural collaborative filtering with FastAI](https://jovian.ml/aakashns/5bc23520933b4cc187cfe18e5dd7e2ed)
  * [Lesson 4: NLP; Tabular data; Collaborative filtering; Embeddings](https://course.fast.ai/videos/?lesson=4) and its [ipynb](https://nbviewer.jupyter.org/github/fastai/course-v3/blob/master/nbs/dl1/lesson4-collab.ipynb))

**2. Lightfm**
 <p align="left">
  <img src="https://github.com/lyst/lightfm/raw/master/lightfm.png"  width="300">
</p>
  
  [LightFM](https://github.com/lyst/lightfm) is a Python implementation of a number of popular recommendation algorithms. It also contains some datasets to try on.

  * [An implicit feedback recommender for the Movielens dataset](https://github.com/lyst/lightfm/tree/master/examples/movielens)

## **Resources**
1. https://grouplens.org/
2. http://surpriselib.com/
3. https://towardsdatascience.com/building-and-testing-recommender-systems-with-surprise-step-by-step-d4ba702ef80b
4. https://blog.codecentric.de/en/2019/07/recommender-system-movie-lens-dataset/
5. https://github.com/NicolasHug/Surprise/blob/master/examples/benchmark.py